In [1]:
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization

from keras.layers import *
from keras.models import *

from keras.regularizers import l2

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
from helpers import TrainingMonitor
from helpers import Utils
from helpers import AttentionDecoder

In [4]:
Xoh_file = open('../input/dataset/Xoh.pickle', 'rb')
x = pickle.load(Xoh_file)
Xoh_file.close()

In [5]:
Yoh_file = open('../input/dataset/Yoh.pickle', 'rb')
y = pickle.load(Yoh_file)
Yoh_file.close()

In [6]:
x[0].shape

(30, 37)

In [7]:
y[0].shape

(10, 11)

In [8]:
train_split, val_split = 0.7, 0.15

train_len = int(train_split * x.shape[0])
train_val_len = int((train_split + val_split) * x.shape[0])

x_train = x[:train_len]
x_val = x[train_len:train_val_len]
x_test = x[train_val_len:]

y_train = y[:train_len]
y_val = y[train_len:train_val_len]
y_test = y[train_val_len:]

In [9]:
x_train.shape

(700, 30, 37)

In [10]:
TIME_STEPS = x_train.shape[1]
INPUT_DIM = x_train.shape[2]

In [11]:
y_train.shape

(700, 10, 11)

In [15]:
model = Sequential()
model.add(LSTM(128, input_shape=(TIME_STEPS, INPUT_DIM)))
model.add(RepeatVector(10))
model.add(AttentionDecoder(128, 11))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               84992     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 10, 128)           0         
_________________________________________________________________
AttentionDecoder (AttentionD (None, 10, 11)            155268    
Total params: 240,260
Trainable params: 240,260
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [18]:
output_path = "../output/"

print(os.getpid())
filepath=output_path + "progress/seq2seq-attention-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

2292


In [19]:
history = model.fit(x_train, y_train,
            batch_size=32,
            epochs=100,
            validation_data=(x_val, y_val),
            shuffle="batch",
            callbacks=callbacks)

Train on 700 samples, validate on 150 samples
Epoch 1/100
700/700 [==============================] - 4s 6ms/step - loss: 2.1033 - acc: 0.2846 - val_loss: 1.8463 - val_acc: 0.3560
Epoch 2/100
700/700 [==============================] - 2s 2ms/step - loss: 1.7508 - acc: 0.3649 - val_loss: 1.6439 - val_acc: 0.3787
Epoch 3/100
700/700 [==============================] - 2s 2ms/step - loss: 1.4250 - acc: 0.4654 - val_loss: 1.2676 - val_acc: 0.4147
Epoch 4/100
700/700 [==============================] - 2s 2ms/step - loss: 1.2124 - acc: 0.4834 - val_loss: 1.2069 - val_acc: 0.4813
Epoch 5/100
700/700 [==============================] - 2s 2ms/step - loss: 1.1907 - acc: 0.4939 - val_loss: 1.1821 - val_acc: 0.4913
Epoch 6/100
700/700 [==============================] - 2s 2ms/step - loss: 1.1787 - acc: 0.4981 - val_loss: 1.1903 - val_acc: 0.4833
Epoch 7/100
700/700 [==============================] - 2s 2ms/step - loss: 1.1772 - acc: 0.4959 - val_loss: 1.1685 - val_acc: 0.4900
Epoch 8/100
700/700 [==

In [20]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 99.43%; Val: 86.47%; Test: 85.93%


In [18]:
model_json = model.to_json()
with open("../output/saved/date_model_seq2seq_94.33.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../output/saved/date_weight_seq2seq_94.33.hdf5")